# Preamble

Analysis of a set of simulation outputs that are the result of a parameter sweep.

## Usage

- Save analysis notebook `lunet_stn-gpe_analysis.ipynb` with cells cleared and hard-coded settings uncommented.


- Edit sweep analysis script `run_notebooks_sweep.py` with correct sweep variable, time interval.


- Run sweep analysis script to execute analysis notebook on all simulation outputs.


- Paste same paths used in script in section 'Load Data'

In [ ]:
%matplotlib inline
import os.path
import cPickle as pickle

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy.stats
from statsmodels.formula.api import ols
import pandas

%load_ext bgcellmodels.extensions.jupyter.skip_cell_extension

## Plotting Options

In [ ]:
# Width of the page for calibrating fig_size.
# Approx. 16 for matplotlib backend %inline  8 for %notebook
if matplotlib.get_backend() == 'nbAgg':
    from bgcellmodels.extensions.jupyter import jupyterutil
    jupyterutil.notebook_show_figs_after_exception() # fix bug for notebook backend where figures not shown
    page_width = 10
else:
    page_width = 10
ax_height = 3

# Style of figures (default colors etc.): see https://matplotlib.org/gallery/style_sheets/style_sheets_reference.html
plt.style.use('default')

## Settings Variables

In [ ]:
# SETPARAM: paste output directories below (e.g use ctrl+c in Nautilus file manager)
outputs_clipboard = """
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/LuNetStnGpe/q4_sweep_gpe-inh-aff-ratio/LuNetStnGpe_2018.12.03_12.28.58_job-1190110.sonic-head_StnGpe_syn-V18_gpe-inh-aff_x-0.70
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/LuNetStnGpe/q4_sweep_gpe-inh-aff-ratio/LuNetStnGpe_2018.12.03_12.29.00_job-1190107.sonic-head_StnGpe_syn-V18_gpe-inh-aff_x-0.40
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/LuNetStnGpe/q4_sweep_gpe-inh-aff-ratio/LuNetStnGpe_2018.12.03_12.29.05_job-1190108.sonic-head_StnGpe_syn-V18_gpe-inh-aff_x-0.50
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/LuNetStnGpe/q4_sweep_gpe-inh-aff-ratio/LuNetStnGpe_2018.12.03_12.29.06_job-1190109.sonic-head_StnGpe_syn-V18_gpe-inh-aff_x-0.60
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/LuNetStnGpe/q4_sweep_gpe-inh-aff-ratio/LuNetStnGpe_2018.12.03_12.29.09_job-1190105.sonic-head_StnGpe_syn-V18_gpe-inh-aff_x-0.20
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/LuNetStnGpe/q4_sweep_gpe-inh-aff-ratio/LuNetStnGpe_2018.12.03_12.29.09_job-1190106.sonic-head_StnGpe_syn-V18_gpe-inh-aff_x-0.30
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/LuNetStnGpe/q4_sweep_gpe-inh-aff-ratio/LuNetStnGpe_2018.12.03_12.29.12_job-1190111.sonic-head_StnGpe_syn-V18_gpe-inh-aff_x-0.80
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/LuNetStnGpe/q4_sweep_gpe-inh-aff-ratio/LuNetStnGpe_2018.12.03_12.29.16_job-1190112.sonic-head_StnGpe_syn-V18_gpe-inh-aff_x-0.90
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/LuNetStnGpe/q4_sweep_gpe-inh-aff-ratio/LuNetStnGpe_2018.12.03_12.29.19_job-1190104.sonic-head_StnGpe_syn-V18_gpe-inh-aff_x-0.10
"""
output_dirs = outputs_clipboard.strip().split()

In [ ]:
# Analysis results for each simulation
analysis_results = {} # dict[sweep_var: float, exported_data: dict]

# SETPARAM: override sweep variable name if desired
sweep_var_name = 'ratio_gpe_msn'
# SETPARAM: description of sweep variable for figure legends
sweep_var_legend = "$gmax_{gpe}/gmax_{msn}$" # LaTeX expressions allowed, double backslashes (\ -> \\)

# SETPARAM: filename of exported pickle file
export_filename = 'analysis_results_1.0s-6.0s_SPONT.pkl' # 'analysis_results_6.0s-10.0s_DRIVEN.pkl'
# SETPARAM: filename for output notebook
report_filename = '{}_sweep-meta_SPONT_V2.html'.format(sweep_var_name)

# SETPARAM: whether cortical bursting input is given
without_ctx_bursting = True

# SETPARAM: Entries in output_dirs with incorrect sweep variable
custom_outputs = { 
#     2 : 1.0,
}
override_sweep_vals = None # np.round(np.arange(1./3, 3+1./3, 1./3), 2)

for i, odir in enumerate(output_dirs):
    export_file = os.path.join(odir, export_filename)
    with open(export_file, 'rb') as f:
        results_dict = pickle.load(f)
        
        if sweep_var_name is None and i == 0:
            sweep_var_name = results_dict['sweep_settings']['var_name']
        
        if i in custom_outputs.keys():
            sweep_value = custom_outputs[i]
        elif override_sweep_vals is not None:
            sweep_value = override_sweep_vals[i]
        else:
            sweep_value = float(results_dict['sweep_settings']['var_value'])
            
#         elif sweep_var_name == results_dict['sweep_var_name']:
#             sweep_value = results_dict['sweep_var_value']
#         else:
#             raise ValueError(
#                 "Simulation results in {} do not have the intended sweep variable!"
#                 "\nMaybe one of the output directories you copied does not match the series.".format(odir))
        
        # Extract sweep value from filename
        # import re
        # match = re.search(r'[0-9]+$', odir)
        # sweep_value = float(match.group(0))
        
        # Use saved sweep value
        analysis_results[sweep_value] = results_dict

In [ ]:
print(sorted(analysis_results.keys()))
print(analysis_results.values()[0].keys())

## Support Functions

In [ ]:
save_fig_path = '/home/luye/cloudstore_m/notes/publications_lkmn/ARTICLE_stn_gpe_loop/figures/sec1_balance_exc_inh'
def save_figure(fname, fig=None, **kwargs):
    """
    Save given or current figure.
    For LaTeX embedding, use extension pdf/pgf/eps.
    """
    if fig is None:
        plt.savefig(fname, **kwargs) # save current figure
    else:
        fig.savefig(fname, **kwargs) # save specific figure

In [ ]:
line_colors = 'crkgbm'
populations = 'CTX', 'STN', 'STN.surrogate', 'GPE.proto', 'GPE.surrogate', 'STR.MSN' # TODO: load from exported vars
pop_color_map = {k: line_colors[i % len(line_colors)] for i,k in enumerate(sorted(populations))}

def get_pop_color(pop_label):
    return pop_color_map[pop_label]

In [ ]:
def integrate_subband(freqs, vals, fband):
    mask1 = (freqs >= fband[0]) & (freqs <= fband[1])
    mask2 = np.isclose(freqs, fband[0], atol=0.1) | np.isclose(freqs, fband[1], atol=0.1)
    fband_idx, = np.where(mask1 | mask2)
    return np.sum(vals[fband_idx])

# Sec 1 - Balance EXC/INH

## S1: Firing Rates

It is useful to plot firing rates to spot differences between the simulations that may underlie certain results.

In [ ]:
# Plot mean firing rates
sweep_vals = np.array(sorted(analysis_results.keys()))

fig, ax = plt.subplots(figsize=(0.8*page_width, ax_height))
ax.set_title('Mean population firing rates ')
plotted = []

for pop in 'STN', 'GPE.proto', 'STR.MSN':
    rates = [analysis_results[val]['mean_rate'][pop] for val in sweep_vals]
    color = get_pop_color(pop)
    plotted.extend(ax.plot(sweep_vals, rates, color=color, label=pop))
    plotted.extend(ax.plot(sweep_vals, rates, color=color, marker='+'))

# Plot experimental rates
invivo_rates = {
    'GPE.proto': {
        'CTL.Rat': [32.4, 35.0], # Sleep-Act.Mallet2008
        'DD.Rat': [13.6, 14.6], # Sleep-Act.Mallet2008
    },
    'STN': {
        'CTL.Rat': [9.8, 16.6], # Sleep-Act.Mallet2008-Kita2011
        'DD.Rat': [17.3, 37.4], # Sleep-Act.Mallet2008-Kita2011
    }
}
for i, pop_label in enumerate(('STN', 'GPE.proto')):
    for j, src_rates in enumerate(invivo_rates.get(pop_label, {}).items()):
        label, bounds = src_rates
        color = get_pop_color(pop_label)
        line_kwargs = {'color': color, 'alpha': 0.2, 'linewidth': 1.0, 
                       'hatch': '\\' if label.startswith('DD') else '/',
                       'label': '{} - {}'.format(pop_label, label)}
        plotted.append(ax.fill_between(sweep_vals, bounds[0], bounds[1], **line_kwargs))
        # ax.hlines(bounds, x0, x1, **line_kwargs)
    
# Axes & Legend
ax.set_xlabel(sweep_var_legend)
ax.set_ylabel('Population firing rate (Hz)')
fig.subplots_adjust(bottom=0.15) # prevent clipped xlabel
ax.set_xticks(sweep_vals)
# # ax.set_xticklabels(sweep_vals)
# # ax.set_yticks(np.arange(0, int(max(rates)+2), 5), minor=False)
# # ax.set_yticks(np.arange(0, int(max(rates)+2), 1.0), minor=True)
ax.grid(True, axis='y')
ax.legend()

# Plot legend in separate figure
# fig, ax = plt.subplots()
# fig.legend(plotted, [p.get_label() for p in plotted], loc='upper center', frameon=False)
# ax.axis('off')

In [ ]:
la = plotted[0]
la.get_label()

## S2: EXC/INH Currents

It is useful to plot total synaptic input currents to spot differences between the simulations that may underlie certain results.

In [ ]:
# Plot mean firing rates
sweep_vals = np.array(sorted(analysis_results.keys()))

fig, ax = plt.subplots(figsize=(0.8*page_width, ax_height))
ax.set_title('Balance of excitatory/inhibitory synaptic currents')

for pop in 'STN', 'GPE.proto':
    rates = [analysis_results[val]['I_exc_inh_ratio'][pop] for val in sweep_vals]
    color = get_pop_color(pop)
    ax.plot(sweep_vals, rates, color=color, label=pop)
    ax.plot(sweep_vals, rates, color=color, marker='+')

ax.set_xlabel(sweep_var_legend)
ax.set_ylabel('Itot_exc / Itot_inh')
fig.subplots_adjust(bottom=0.15) # prevent clipped xlabel


ax.set_xticks(sweep_vals)
ax.grid(True, axis='y')
ax.legend()

## S3: Afferent Currents

Ratio of afferent currents for each population

In [ ]:
def plot_afferent_currents(pop_label):
    """
    Plot ratio of afferent currents as filled region under graph.
    """
    if 'I_afferents' not in analysis_results.values()[0]:
        print("Detailed afferent currents info not saved. Skipping plot.")
        return
    
    fig, ax = plt.subplots(figsize=(.75*page_width, 1.25*ax_height))
    ax.set_title('{} total afferent currents'.format(pop_label))
    leg_actors = [] # actors in figure that will get legend
    
    # Total current for each afferent as array for each sweep value
    # NOTE: total only over sampled cells, not entire population
    sweep_vals = np.array(sorted(analysis_results.keys()))
    sweep_itot_data = [analysis_results[val]['I_afferents'][pop_label] for val in sweep_vals]
    pops_sweep_itot = {aff: [] for aff in sweep_itot_data[0][0].keys() if aff not in ('EXC', 'INH')}
    
    itot_neg_cumul = np.zeros_like(sweep_vals)
    itot_pos_cumul = np.zeros_like(sweep_vals)
    itot_neg_lines = [itot_neg_cumul]
    itot_pos_lines = [itot_pos_cumul]
    itot_neg_vals = []
    itot_pos_vals = []
    
    for pop in pops_sweep_itot.keys():
        sweep_itot = np.array([sum((sum(cell_iaff[pop].values()) for cell_iaff in itot_data)) 
                                 for itot_data in sweep_itot_data])
    
        if sweep_itot.mean() > 0: # inhibitory current -> make negative
            itot_neg_vals.append(sweep_itot)
            itot_neg_lines.append(itot_neg_lines[-1] - sweep_itot)
            ax.plot(sweep_vals, itot_neg_lines[-1], color='k')
            actors = ax.fill_between(sweep_vals, itot_neg_lines[-2], itot_neg_lines[-1],
                                     label=pop, alpha=0.5)
        else:                    # excitatory current -> make positive
            itot_pos_vals.append(sweep_itot)
            itot_pos_lines.append(itot_pos_lines[-1] - sweep_itot)
            ax.plot(sweep_vals, itot_pos_lines[-1], color='k')
            actors = ax.fill_between(sweep_vals, itot_pos_lines[-2], itot_pos_lines[-1],
                                     label=pop, alpha=0.5)
        leg_actors.append(actors)
    
    # Center line
    ax.plot(sweep_vals, itot_pos_lines[0], color='k')
    
    # Plot sum of all currents
    itot_all_cumul = -np.sum(itot_pos_vals, axis=0) - np.sum(itot_neg_vals, axis=0)
    leg_actors.extend(
        ax.plot(sweep_vals, itot_all_cumul, 'r--', label='sum'))
    
    # Plot ratio
    ax2 = ax.twinx()
    itot_all_ratio = -np.sum(itot_pos_vals, axis=0) / np.sum(itot_neg_vals, axis=0)
    leg_actors.extend(
        ax2.plot(sweep_vals, itot_all_ratio, 'm--', label='ratio'))
    ax2.set_ylabel('ratio $i_{EXC}$ / $i_{INH}$')
    ratio_lim = np.abs(itot_all_ratio).max()
    ax2.set_ylim((-1.1*ratio_lim, 1.1*ratio_lim))
    
    ax.set_xticks(sweep_vals)
    ax.set_ylabel('current (nA)')
    ax.set_xlabel(sweep_var_legend)
    ax.grid(True)
    ax.legend(leg_actors, [a.get_label() for a in leg_actors], loc='lower right')
    fig.subplots_adjust(bottom=0.15) # prevent clipped xlabel

In [ ]:
plot_afferent_currents('STN')
plot_afferent_currents('GPE.proto')

## 1A : PSD vs EXC/INH ratio

### PSD 'sweep-gram'

In [ ]:
def compare_psd_heatmap(sig_label, x_axis_equidistant=True, Pmax=None):
    """
    Compare PSD across parameter sweep using heatmap visualization.
    
    @param  x_axis_equidistant : bool
            If true, x axis will be spaces equidistanctly, and missing values
            will be filled by black bands.
    """
    
    # Concatenate PSDs of different sweep values into matrix
    test_freqs, test_psd = analysis_results.values()[0]['PSD'][sig_label]
    freq_res = test_freqs[1] - test_freqs[0]
    fmax = 100.0
    nfreq = int(fmax/freq_res) + 1

    # Sweep values may not be equidistantly spaced
    sweep_vals = np.array(sorted(analysis_results.keys()))
    d_sweep = min(np.diff(sweep_vals)) # resolution of sweep variable
    sweep_axis_spaced = np.arange(sweep_vals.min(), sweep_vals.max()+d_sweep, d_sweep)
    sweep_axis_full = list(sweep_axis_spaced)
    for v in sweep_vals: # steps may miss actual values
        if not np.any(np.isclose(v, sweep_axis_full)):
            sweep_axis_full.append(v)
    if x_axis_equidistant:
        sweep_axis = np.array(sorted(sweep_axis_full))
    else:
        sweep_axis = sweep_vals

    # Cronstruct image (heatmap) matrix
    freq_axis = test_freqs[:nfreq]
    sweep_psds = np.empty((nfreq, sweep_axis.size), dtype=float)
    valid_columns = []
    for sweep_value, results in analysis_results.iteritems():
        # col_id = int((sweep_value - sweep_axis.min())/d_sweep)
        col_id = np.where(np.isclose(sweep_value, sweep_axis))[0][0]
        valid_columns.append(col_id)
        freqs, psd = results['PSD'][sig_label]
        sweep_psds[:, col_id] = psd.magnitude[:nfreq]

    # Fill columns for missing sweep values with NaN
    invalid_columns = [i for i in range(sweep_axis.size) if i not in valid_columns]
    sweep_psds[:, invalid_columns] = np.nan
    psd_sweep = np.ma.masked_array(sweep_psds, np.isnan(sweep_psds)) # mark invalid/missing data

    # Draw heat map
    cmap = plt.get_cmap('viridis') # e.g. jet, plasma: https://matplotlib.org/examples/color/colormaps_reference.html
    cmap.set_bad(color='k') # color for masked values

    fig, ax = plt.subplots(figsize=(0.8*page_width, ax_height))
    sweep_plotaxis = np.concatenate((sweep_axis, [sweep_axis.max()+d_sweep])) # fix bug/feature in pcolormesh
    plt.pcolormesh(sweep_plotaxis, freq_axis, psd_sweep, cmap=cmap)
    if Pmax is not None:
        plt.clim(0, Pmax)

    # f_max = 50
    # plt.ylim((0, f_max))
    plt.colorbar()
    
    # cmap.set_bad('grey')
    ax.set_xticks(sweep_axis + d_sweep/2.0) # otherwise they appear left of column
    ax.set_xticklabels(['{:.2f}'.format(v) for v in sweep_axis])
    ax.set_xlim((sweep_axis.min(), sweep_plotaxis.max()))
    ax.set_yticks(np.arange(0,50,1), minor=True)
    ax.set_yticks(np.arange(0,55,5), minor=False)
    ax.set_ylim((0, 50))
    # TODO: find out why one column doesn't show

    plt.ylabel('frequency (Hz)')
    plt.xlabel(sweep_var_legend)
    plt.suptitle('Evolution of PSD ({})'.format(sig_label))
    fig.subplots_adjust(bottom=0.15) # prevent clipped xlabel

In [ ]:
compare_psd_heatmap('STN_Vm', False)
compare_psd_heatmap('STN_Vm', False, Pmax=0.5)
compare_psd_heatmap('GPE.proto_Vm', False)

### [alt] PSDs superimposed

In [ ]:
# Plot the PSD
sig_label = 'STN_Vm'
sweep_vals = np.array(sorted(analysis_results.keys()))

# Sampling colors from linear scale:
# https://stackoverflow.com/questions/8931268/using-colormaps-to-set-color-of-line-in-matplotlib
cmap = plt.get_cmap('cool')

# Line colors equidistantly on color scale, independent of sweep values
line_colors = [cmap(x) for x in np.linspace(0, 1, len(analysis_results))]
def get_line_color(sweep_val):
    return cmap(float(sweep_val) / sweep_vals.max)

# Linear map of sweep values to color scale
cNorm  = matplotlib.colors.Normalize(vmin=sweep_vals[0], vmax=sweep_vals[-1])
scalarMap = matplotlib.cm.ScalarMappable(norm=cNorm, cmap=cmap)
def get_line_color(sweep_val):
    return scalarMap.to_rgba(sweep_val)

def compare_psd(sig_label, legend=False, colorbar=True, width=page_width, height=1.2*ax_height):
    """ Plot PSD for given signal for each simulation / sweep variable. """
    if colorbar:
        ncols = 2
        # axes for colorbar must be smaller
        subplot_kwargs = {'gridspec_kw': {'width_ratios':[20, 1]}}
    else:
        ncols = 1
        subplot_kwargs = {}
    
    fig, axes = plt.subplots(1, ncols, figsize=(width, height), **subplot_kwargs)
    if colorbar:
        ax = axes[0]
    else:
        ax = axes
    
    # Plot PSD as line for  each sweep value
    sweep_vals = np.array(sorted(analysis_results.keys()))
    lines = []
    for i, sweep_value in enumerate(sweep_vals):
        results = analysis_results[sweep_value]
        freqs, psd = results['PSD'][sig_label]
        plotted = ax.plot(freqs, psd,
                          color=get_line_color(sweep_value), 
                          label='{} = {}'.format(sweep_var_legend, sweep_value))
        lines.extend(plotted)
    
    ax.set_ylabel('Power ({})'.format(psd.units))
    ax.set_xlabel('frequency ({})'.format(freqs.units))
    ax.set_xticks(np.arange(0,55,5), minor=False)
    ax.set_xticks(np.arange(0,51,1), minor=True)
    ax.grid(True)
    ax.set_xlim((0, 35))
    # ax.set_yscale('log')
    ax.set_title('Welch PSD for {}'.format(sig_label))
    if legend:
        if len(sweep_vals) < 7:
            ax.legend(loc='upper right', bbox_to_anchor=(1,1))
        else:
            lower = slice(7)
            upper = slice(7, len(sweep_vals))
            labels = ['{} = {}'.format(sweep_var_legend, v) for v in sweep_vals]
            # 'loc' is corner of legend anchored to anchorpoint
            # 'bbox_to_anchor' is location of anchorpoint on axis bounding box
            leg1 = ax.legend(lines[lower], labels[lower], loc=1, bbox_to_anchor=(.82,1))
            leg2 = ax.legend(lines[upper], labels[upper], loc=2, bbox_to_anchor=(.82,1))
            ax.add_artist(leg1) # otherwise last legend is used
    if colorbar:
        ax = axes[1]
        # https://matplotlib.org/api/_as_gen/matplotlib.pyplot.colorbar.html
        cb = matplotlib.colorbar.ColorbarBase(ax, cmap=cmap, norm=cNorm, ticks=sweep_vals,
                                              orientation='vertical')
        cb.set_label(sweep_var_legend)
    fig.subplots_adjust(bottom=0.15) # prevent clipped xlabel

In [ ]:
compare_psd('STN_Vm')
compare_psd('GPE.proto_Vm')

### [alt] PSD sum (sub) beta bands

Summed PSD power in different frequency bands.

In [ ]:

def psd_power_integrated(sig_label, f_band, detailed=True, independent='sweep', **kwargs):
    """
    Linear regression of integrated composite coherence.
    
    @param    independent : str
              Independent variable for plot x-axis and linear regression
    
    @param    x_label : str
              label for independent variable
    
    @param    x_ticks : list(float)
              Axes ticks for x-axis
    """
    sweep_vals = np.array(sorted(analysis_results.keys()))
    
    all_Psum = []
    for i, sweep_value in enumerate(sweep_vals):
        f, Pxx = analysis_results[sweep_value]['PSD'][sig_label]
        all_Psum.append(integrate_subband(f.magnitude, Pxx.magnitude, f_band))
    
    # Set independent variable for regression
    if independent == 'sweep':
        x = sweep_vals
        x_label = sweep_var_legend
        x_ticks = sweep_vals
    elif independent.lower() == 'exc_inh_ratio':
        x = np.array([analysis_results[val]['I_exc_inh_ratio'][pop] for val in sweep_vals])
        x_label = '$I_{exc} / I_{inh}$'
        x_ticks = kwargs.get('x_ticks', None)
    elif independent == 'custom':
        x = kwargs['x']
        x_label = kwargs['x_label']
        x_ticks = kwargs.get('x_ticks', None)
    else:
        raise ValueError("Argument <independent> must be either 'sweep', 'exc_inh_ratio', or 'custom'.")
    
    # Linear regression
    metric_name = 'Pxx'
    y = np.array(all_Psum)
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)

    # Detailed linear regression
    if detailed:
        data = pandas.DataFrame({'x': x, metric_name: y})
        model = ols("{} ~ {}".format(metric_name, 'x'), data).fit()
        print(model.summary()) # summary2()

    fig, ax = plt.subplots(figsize=(.75*page_width, 1.25*ax_height))
    ax.set_title('{} : mean(PSD) between {} - {} Hz'.format(sig_label, f_band[0], f_band[1]))
    ax.plot(x, y, 'o', color='g', label='original data')
    ax.plot(x, intercept + slope*x, 'k--', label='linear fit')
    if x_ticks is not None:
        ax.set_xticks(x_ticks)
    ax.set_ylabel('Power ($mV^2/Hz$)')
    ax.set_xlabel(x_label)
    ax.grid(True)
    ax.text(.98, .05, '$R^2$ = {:.2f}\n$p^r$ = {:f}'.format(r_value**2, p_value), 
            transform=ax.transAxes, ha='right')
    fig.subplots_adjust(bottom=0.15) # prevent clipped xlabel
    

In [ ]:
psd_power_integrated('STN_Vm', [0.,12.], detailed=False)
psd_power_integrated('STN_Vm', [6.,12.], detailed=False)
psd_power_integrated('STN_Vm', [13., 20.], detailed=False)
psd_power_integrated('STN_Vm', [20.,30.], detailed=False)

psd_power_integrated('GPE.proto_Vm', [0.,12.], detailed=False)
psd_power_integrated('GPE.proto_Vm', [6.,12.], detailed=False)
psd_power_integrated('GPE.proto_Vm', [13., 20.], detailed=False)
psd_power_integrated('GPE.proto_Vm', [20.,30.], detailed=False)

### [alt] PSD sum around peak

Sum PSD power in frequency bins around the peak.

In [ ]:
# SETPARAM: applied frequency of oscillatory bursts
f_burst = 25.0
psd_power_integrated('STN_Vm', [f_burst-2, f_burst+2], 
                     detailed=False, independent='exc_inh_ratio')
psd_power_integrated('GPE.proto_Vm', [f_burst-2, f_burst+2],
                     detailed=False, independent='exc_inh_ratio')

In [ ]:

def psd_power_integrated(sig_label, f_band, detailed=True):
    """
    Linear regression of integrated composite coherence.
    """
    sweep_vals = np.array(sorted(analysis_results.keys()))
    
    all_Psum = []
    for i, sweep_value in enumerate(sweep_vals):
        f, Pxx = analysis_results[sweep_value]['PSD'][sig_label]
        all_Psum.append(integrate_subband(f.magnitude, Pxx.magnitude, f_band))
    
    # Linear regression
    metric_name = 'Pxx'
    x = sweep_vals
    y = np.array(all_Psum)
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)

    # Detailed linear regression
    if detailed:
        data = pandas.DataFrame({'x': x, metric_name: y})
        model = ols("{} ~ {}".format(metric_name, 'x'), data).fit()
        print(model.summary()) # summary2()

    fig, ax = plt.subplots(figsize=(.75*page_width, 1.25*ax_height))
    ax.set_title('{} : mean(PSD) between {} - {} Hz'.format(sig_label, f_band[0], f_band[1]))
    ax.plot(x, y, 'o', color='g', label='original data')
    ax.plot(x, intercept + slope*x, 'k--', label='linear fit')
    ax.set_xticks(sweep_vals)
    ax.set_ylabel('Power ($mV^2/Hz$)')
    ax.set_xlabel(sweep_var_legend)
    ax.grid(True)
    ax.text(.98, .05, '$R^2$ = {:.2f}\n$p^r$ = {:f}'.format(r_value**2, p_value), 
            transform=ax.transAxes, ha='right')
    fig.subplots_adjust(bottom=0.15) # prevent clipped xlabel
    

### [alt] PSD Beta peak

Find frequency of largest peak within Beta range.

In [ ]:
def compare_Pmax_frequency(sig_labels, fband):
    """
    Find frequency of largest peak within Beta range.
    """
    fig, ax = plt.subplots(figsize=(0.75*page_width, ax_height))
    ax.set_title('frequency of Pmax in {} - {} Hz'.format(fband[0], fband[1]))
    
    sweep_vals = np.array(sorted(analysis_results.keys()))
    
    colors = 'bgcm'
    for i, sig_label in enumerate(sig_labels):
        freqs_Pmax = []
        for sweep_value in sweep_vals:
            f, Pxx = analysis_results[sweep_value]['PSD'][sig_label]
            
            # Calculate frequency of Pmax
            freqs = f.magnitude
            mask1 = (freqs >= fband[0]) & (freqs <= fband[1])
            mask2 = np.isclose(freqs, fband[0], atol=0.1) | np.isclose(freqs, fband[1], atol=0.1)
            fband_mask = mask1 | mask2
            Pxx_fband = Pxx[fband_mask]
            freqs_fband = freqs[fband_mask]
            f_pmax = freqs_fband[Pxx_fband == Pxx_fband.max()][0]
            freqs_Pmax.append(f_pmax)
        
        # Plot frequencies
        ax.plot(sweep_vals, freqs_Pmax, '-', 
                color=get_pop_color(sig_label[:-3]), 
                label="$f_{Pmax}$" + " ({sig})".format(sig=sig_label))
        ax.plot(sweep_vals, freqs_Pmax, 'r+')

    ax.set_xticks(sweep_vals)
    ax.set_ylabel('frequency (Hz)')
    ax.set_xlabel(sweep_var_legend)
    ax.grid(True)
    ax.legend()
    fig.subplots_adjust(bottom=0.15) # prevent clipped xlabel

In [ ]:
compare_Pmax_frequency(('STN_Vm', 'GPE.proto_Vm'), (13., 30.))

### [alt] max spectrogram peak

Note that this might give a different picture than comparing simulations with PSD alone: a simulation with periodic synchronization may have higher transient peaks, whereas the PSD (average over time) is lower.

In [ ]:
# (freqs[0:int(50/df)], t, Sxx[:,0:int(50/df)])
# Syy = Sxx[:, int(5000.0/t_res):]

# set temporal resolution of PSD from value in synchrony_analysis_auto.ipynb
# t_res = 20.0 # ms

def compare_max_spectrogram(sig_label, band=(4.0, 30.0), t_start=5000.0):
    """ Plot magnitude of largest spectrogram peak """
    fig, ax = plt.subplots(figsize=(0.75*page_width, ax_height))
    sweep_vals = np.array(sorted(analysis_results.keys()))
    sweep_power = []
    for i, sweep_value in enumerate(sweep_vals):
        results = analysis_results[sweep_value]
        freqs, t, Sxx = results['spectrogram'][sig_label]
        # only look at power in sub-band
        df = freqs[1] - freqs[0]
        f_slice = np.s_[int(band[0]/df):(int(band[1]/df)+1)]
        # first 5000 ms already cut when saving
        # dt = t[1] - t[0]
        # t_slice = np.s_[int(t_start/dt):]
        t_slice = np.s_[:]
        S_max = Sxx[f_slice, t_slice].max()
        sweep_power.append(S_max)
    
    ax.plot(sweep_vals, sweep_power, '-')
    ax.plot(sweep_vals, sweep_power, 'r+')
    # ax.set_xticks(np.arange(0,5*(sweep_vals.max()//5+1),5), minor=False)
    # ax.set_xticks(np.arange(0,1*(sweep_vals.max()//1+1),1), minor=True)
    # ax.set_xlim((0, 50))
    ax.set_ylim((0, 1.1*max(sweep_power)))
    ax.set_ylabel('Power (mV^2/Hz)')
    ax.set_xlabel(sweep_var_legend)
    ax.grid(True)
    ax.set_title('Spectrogram max peak ({})'.format(sig_label))
    fig.subplots_adjust(bottom=0.15) # prevent clipped xlabel

In [ ]:
# compare_max_spectrogram('STN_Vm')
# compare_max_spectrogram('GPE_Vm')

## 1B: Burst Metrics

In [ ]:

def gather_metrics(all_cell_metrics, metric):
    """
    Gather burst metric from all cells into a list
    """
    if isinstance(all_cell_metrics[0][metric], float):
        map_func = lambda x,y: x + [y]
    else:
        map_func = lambda x,y: x + y
    all_cell_vals = reduce(map_func, (cell_metrics[metric] for cell_metrics in all_cell_metrics), [])
    return all_cell_vals


def compare_burst_metrics(pop_label, metric_names=None):
    """
    Compare burst metrics for same population across sweep values
    """
    sweep_vals = np.array(sorted(analysis_results.keys()))
    if metric_names == None:
        metric_names = analysis_results[sweep_vals[0]]['burst_metrics'][pop_label][0].keys()
    
    sweep_metrics = {}
    for metric_name in metric_names:
        for i, sweep_value in enumerate(sweep_vals):
            cell_metrics = analysis_results[sweep_value]['burst_metrics'][pop_label]
            all_cell_vals = gather_metrics(cell_metrics, metric_name)
            sweep_metrics.setdefault(metric_name, []).append(all_cell_vals)
    
        # Plot boxplots
        fig, ax = plt.subplots(figsize=(0.75*page_width, ax_height))
        ax.set_title('{} {}'.format(pop_label, metric_name))
        bp = ax.boxplot(sweep_metrics[metric_name], 0, 'g+')
        ax.set_xticklabels(sweep_vals)
        ax.set_ylim((0, ax.get_ylim()[1]))
        ax.set_ylabel(metric_name)
        ax.set_xlabel(sweep_var_legend)
        ax.grid(True, which='major', axis='y')
        fig.subplots_adjust(bottom=0.15) # prevent clipped xlabel


def regression_burst_metrics(pop_label, metric_names=None, detailed=True):
    """
    Linear regression of burst metrics vs sweep variable.
    """
    sweep_vals = np.array(sorted(analysis_results.keys()))
    if metric_names == None:
        metric_names = analysis_results[sweep_vals[0]]['burst_metrics'][pop_label][0].keys()
    
    sweep_metrics = {}
    for metric_name in metric_names:
        for i, sweep_value in enumerate(sweep_vals):
            cell_metrics = analysis_results[sweep_value]['burst_metrics'][pop_label]
            all_cell_vals = gather_metrics(cell_metrics, metric_name)
            sweep_metrics.setdefault(metric_name, []).append(all_cell_vals)
        
        # Linear regression
        x = sweep_vals
        y = [np.median(mvals) for mvals in sweep_metrics[metric_name]]
        slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
        
        # Detailed linear regression
        if detailed:
            data = pandas.DataFrame({'x': x, metric_name: y})
            model = ols("{} ~ {}".format(metric_name, 'x'), data).fit()
            print(model.summary()) # summary2()
        
        fig, ax = plt.subplots(figsize=(0.75*page_width, 1.5*ax_height))
        ax.set_title('{} {}'.format(pop_label, metric_name))
        ax.set_xticks(sweep_vals)
        ax.plot(x, y, 'o', color='g', label='original data')
        ax.plot(x, intercept + slope*x, 'k--', label='linear fit')
        ax.set_ylabel(metric_name)
        ax.set_xlabel(sweep_var_legend)
        ax.grid(True)
        ax.text(.98, .05, '$R^2$ = {:.2f}\n$p^r$ = {:f}'.format(r_value**2, p_value), 
                transform=ax.transAxes, ha='right')
        fig.subplots_adjust(bottom=0.15) # prevent clipped xlabel
    

### STN Boxplots of Burst Metrics

One boxplot per sweep variable.

In [ ]:
compare_burst_metrics('STN')

### GPE Boxplots of Burst Metrics

One boxplot per sweep variable.

In [ ]:
compare_burst_metrics('GPE.proto')

### [alt] STN Regression Burst metrics

Linear regression & Pearson correlation.

In [ ]:
regression_burst_metrics('STN', detailed=False)

### [alt] GPE Regression Burst metrics

Linear regression & Pearson correlation.

In [ ]:
regression_burst_metrics('GPE.proto', detailed=False)

## 1C: Population Synchronization

In [ ]:
def composite_coherence_integrated(pops, f_band, detailed=True):
    """
    Linear regression of integrated composite coherence.
    """
    sweep_vals = np.array(sorted(analysis_results.keys()))
    if len(pops) > 1:
        pop_a, pop_b = pops
        key = '{}-{}'.format(pop_a, pop_b)
    else:
        key = pops[0]
    
    all_Cxy_sum = []
    for i, sweep_value in enumerate(sweep_vals):
        coherence = analysis_results[sweep_value]['coherence_composite'][key]
        f, Cxy = coherence[0], coherence[1]
        Csum = integrate_subband(f, Cxy, f_band)
        all_Cxy_sum.append(Csum)
    
    # Linear regression
    metric_name = 'Cxy_sum'
    x = sweep_vals
    y = np.array(all_Cxy_sum)
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)

    # Detailed linear regression
    if detailed:
        data = pandas.DataFrame({'x': x, metric_name: y})
        model = ols("{} ~ {}".format(metric_name, 'x'), data).fit()
        print(model.summary()) # summary2()

    fig, ax = plt.subplots(figsize=(.75*page_width, 1.25*ax_height))
    ax.set_title('{} Composite spiketrain coherence ({} - {} Hz)'.format(key, f_band[0], f_band[1]))
    ax.plot(x, y, 'o', color='g', label='original data')
    ax.plot(x, intercept + slope*x, 'k--', label='linear fit')
    ax.set_xticks(sweep_vals)
    ax.set_ylabel(metric_name)
    ax.set_xlabel(sweep_var_legend)
    ax.grid(True)
    ax.text(.98, .05, '$R^2$ = {:.2f}\n$p^r$ = {:f}'.format(r_value**2, p_value), 
            transform=ax.transAxes, ha='right')
    fig.subplots_adjust(bottom=0.15) # prevent clipped xlabel

### STN-GPe Composite Coherence

Integrated coherence in 13-30 Hz frequency band.

In [ ]:
# composite_coherence_integrated('STN', 'GPE.proto', [13.,30.], detailed=False)
composite_coherence_integrated(('STN', 'GPE.proto'), [0.,12.], detailed=False)
composite_coherence_integrated(('STN', 'GPE.proto'), [13.,30.], detailed=False)
composite_coherence_integrated(('STN', 'GPE.proto'), [13.,20.], detailed=False)
composite_coherence_integrated(('STN', 'GPE.proto'), [20.,30.], detailed=False)

### STN Composite Coherence

Integrated coherence in 13-30 Hz frequency band.

In [ ]:
# composite_coherence_integrated('STN', 'GPE.proto', [13.,30.], detailed=False)
composite_coherence_integrated(('STN',), [0.,12.], detailed=False)
composite_coherence_integrated(('STN',), [13.,30.], detailed=False)
composite_coherence_integrated(('STN',), [13.,20.], detailed=False)
composite_coherence_integrated(('STN',), [20.,30.], detailed=False)

### GPE Composite Coherence

Integrated coherence in 13-30 Hz frequency band.

In [ ]:
# composite_coherence_integrated('STN', 'GPE.proto', [13.,30.], detailed=False)
composite_coherence_integrated(('GPE.proto',), [0.,12.], detailed=False)
composite_coherence_integrated(('GPE.proto',), [13.,30.], detailed=False)
composite_coherence_integrated(('GPE.proto',), [13.,20.], detailed=False)
composite_coherence_integrated(('GPE.proto',), [20.,30.], detailed=False)

### [alt] STN Covariance Complexity

In [ ]:
def compare_covariance_complexity(sig_label, detailed=False):
    """
    Compare distribution of Morgera-index values for time segments 
    in simulation.
    """
    sweep_vals = np.array(sorted(analysis_results.keys()))
    M_datasets = []
    M_intervals = []
    for i, sweep_value in enumerate(sweep_vals):
        t, M = analysis_results[sweep_value]['Morgera_index'][sig_label]
        M_datasets.append(M)
        M_intervals.append(t)

    M_array = np.array(M_datasets)
    mean_M = np.mean(M_array, axis=1)
    std_M = np.std(M_array, axis=1)
    med_M = np.median(M_array, axis=1)
    ival_width = M_intervals[0][0][1] - M_intervals[0][0][0]
    
    # Plot boxplots
    fig, ax = plt.subplots(figsize=(0.75*page_width, ax_height))
    ax.set_title('Covariance complexity over {} ms intervals'.format(ival_width))
    bp = ax.boxplot(M_datasets, 0, 'g+')
    ax.set_xticklabels(sweep_vals)
    # ax.set_ylim((0, 1))
    ax.set_ylabel('M (0-1)')
    ax.set_xlabel(sweep_var_legend)
    ax.grid(True, which='major', axis='y')
    fig.subplots_adjust(bottom=0.15) # prevent clipped xlabel
    
    # Linear regression
    metric_name = 'M'
    x = sweep_vals
    y = med_M
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)

    # Detailed linear regression
    if detailed:
        data = pandas.DataFrame({'x': x, metric_name: y})
        model = ols("{} ~ {}".format(metric_name, 'x'), data).fit()
        print(model.summary()) # summary2()

    # Plot linear regression
    fig, ax = plt.subplots(figsize=(.75*page_width, 1.25*ax_height))
    ax.set_title('Median covariance complexity over {} ms intervals'.format(ival_width))
    ax.plot(x, y, 'o', color='g', label='original data')
    ax.plot(x, intercept + slope*x, 'k--', label='linear fit')
    ax.set_xticks(sweep_vals)
    ax.set_ylabel('M (0-1)')
    ax.set_xlabel(sweep_var_legend)
    ax.grid(True)
    ax.text(.98, .05, '$R^2$ = {:.2f}\n$p^r$ = {:f}'.format(r_value**2, p_value), 
            transform=ax.transAxes, ha='right')
    fig.subplots_adjust(bottom=0.15) # prevent clipped xlabel

In [ ]:
compare_covariance_complexity('STN_Vm')

### [alt] GPE Covariance Complexity

In [ ]:
compare_covariance_complexity('GPE.proto_Vm')

## 1C : Phase Locking Strength

Phase locking to input burst frequency. 

**Guidelines**: Notebooks must be executed with time segment containing Beta inputs and correct filtering settings (around Beta frequency).

In [ ]:
def compare_phases_polar(pop_label):
    """
    Compare population phase vectors across simulations (sweep values)
    by plotting a polar plot for each simulation.
    """
    fig = plt.figure(figsize=(0.75*page_width, 0.75*page_width))
    fig.suptitle('Mean angle and vector length of {} neurons'.format(pop_label))
    # fig.suptitle(sweep_var_legend, color='blue', x=0.5, y=0.02)
    
    nplot = len(sweep_vals)
    ncol = 3
    nrow = int(np.ceil(float(nplot) / ncol))
    
    all_cell_vecs = [analysis_results[val]['cell_phase_vecs'][pop_label] for val in sweep_vals]
    all_pop_vecs = [analysis_results[val]['pop_phase_vecs'][pop_label] for val in sweep_vals]
    
    for i, sweep_val in enumerate(sweep_vals):
        cell_vecs = all_cell_vecs[i]
        pop_vec = all_pop_vecs[i]
        
        vec_angs = np.angle(cell_vecs)
        vec_lens = np.abs(cell_vecs)
        rmax = 0.5 # (vec_lens.max() // 0.1 + 1) * 0.1
        
        ax = fig.add_subplot(nrow, ncol, i+1, projection='polar')
        ax.set_title('{}'.format(sweep_val), color='blue', loc='left', ha='left')
        
        ax.vlines(vec_angs, 0, vec_lens, color='g', linewidth=1, snap=True)
        ax.vlines(np.angle(pop_vec), 0, np.abs(pop_vec), color='r', linewidth=3)
        ax.plot(vec_angs, np.zeros_like(vec_angs)+rmax, 'go', markersize=5)

        ax.grid(True)
        ax.set_rticks(np.arange(0.1, 1.1, 0.1)) # less radial ticks
        ax.set_rmax(rmax)
        # ax.set_rlabel_position(-90.0)  # Move radial labels away from plotted line


def regress_vector_length(pop_label, detailed=True):
    """
    Linear regression of population vector length.
    """
    sweep_vals = np.array(sorted(analysis_results.keys()))
    all_pop_vecs = np.array([analysis_results[val]['pop_phase_vecs'][pop_label] for val in sweep_vals])
    all_vec_len = np.abs(all_pop_vecs)
    
    # Linear regression
    metric_name = 'vec_len'
    x = sweep_vals
    y = all_vec_len
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)

    # Detailed linear regression
    if detailed:
        data = pandas.DataFrame({'x': x, metric_name: y})
        model = ols("{} ~ {}".format(metric_name, 'x'), data).fit()
        print(model.summary()) # summary2()

    fig, ax = plt.subplots(figsize=(.75*page_width, 1.25*ax_height))
    ax.set_title('{} phase vector length'.format(pop_label))
    ax.plot(x, y, 'o', color='g', label='original data')
    ax.plot(x, intercept + slope*x, 'k--', label='linear fit')
    ax.set_xticks(sweep_vals)
    ax.set_ylabel('length (0-1)')
    ax.set_xlabel(sweep_var_legend)
    ax.grid(True)
    ax.text(.98, .05, '$R^2$ = {:.2f}\n$p^r$ = {:f}'.format(r_value**2, p_value), 
            transform=ax.transAxes, ha='right')
    fig.subplots_adjust(bottom=0.15) # prevent clipped xlabel

    
def vector_angle_difference(pop_a, pop_b):
    """
    Plot difference in angle of the population vector.
    """
    sweep_vals = np.array(sorted(analysis_results.keys()))
    pops = pop_a, pop_b
    
    pops_vecs = [np.array([analysis_results[val]['pop_phase_vecs'][pop] for val in sweep_vals])
                    for pop in pops]
    pops_angles = [np.angle(vecs, deg=False) for vecs in pops_vecs]
    pops_veclens = [np.abs(vecs) for vecs in pops_vecs]
    pop_angles_diffs = np.unwrap((pops_angles[1] - pops_angles[0]) % (2*np.pi)) * 180/np.pi
     
    fig, ax = plt.subplots(figsize=(.75*page_width, 1.25*ax_height))
    ax.set_title('{}-{} mean phase difference'.format(pop_b, pop_a))
    
    # Plot angle difference on left axis
    ax.plot(sweep_vals, pop_angles_diffs, 'o', color='blue', label='$\Delta\phi$')
    ax.plot(sweep_vals, pop_angles_diffs, '-', color='blue', linewidth=1.0)
    ax.set_ylim((0, max(90, pop_angles_diffs.max()*1.1)))
    ax.set_ylabel('angle (degrees)')
    ax.legend(loc='upper left')
    
    # Plot vector lengths on right axis
    ax2 = ax.twinx()
    for i, pop in enumerate(pops):
        ax2.plot(sweep_vals, pops_veclens[i], color=get_pop_color(pop), label='|{}|'.format(pop))
    ax2.set_ylabel('length (0-1)')
    ax2.legend(loc='upper right')
    ax2.set_ylim((0, max(0.5, 1.1*max(lens.max() for lens in pops_veclens))))
    
    ax.set_xticks(sweep_vals)
    ax.set_xlabel(sweep_var_legend)
    ax.grid(True)
    fig.subplots_adjust(bottom=0.15) # prevent clipped xlabel

### STN Vector Length

Vector length measure: strength of phase locking to Beta input.

In [ ]:
compare_phases_polar('STN')
regress_vector_length('STN', detailed=False)

### GPE Vector Length

Vector length measure: strength of phase locking to Beta input.

In [ ]:
compare_phases_polar('GPE.proto')
regress_vector_length('GPE.proto', detailed=False)

### STN-GPE Angle vs Length

In [ ]:
# TODO take population vector angle module 360, plot on one axis, plot vector length and psd power on another axis
vector_angle_difference('STN', 'GPE.proto')

### [alt] : magnitude PSD @ f_burst
PSD peak magnitude at frequency of applied cortical bursting inputs.

In [ ]:
def compare_psd_magnitude(sig_label, input_freqs):
    """
    Plot PSD magnitude at frequency of applied cortical bursting
    """
    fig, ax = plt.subplots(figsize=(0.75*page_width, ax_height))
    sweep_vals = np.array(sorted(analysis_results.keys()))
    sweep_power = []
    for i, sweep_value in enumerate(sweep_vals):
        results = analysis_results[sweep_value]
        freqs, psd = results['PSD'][sig_label]
        freq_axis = freqs.magnitude
        f_input = input_freqs[i]
        i_entrain, = np.where(freq_axis == f_input)
        p_entrain = psd[i_entrain[0]]
        sweep_power.append(p_entrain)
    
    ax.plot(sweep_vals, sweep_power, '-')
    ax.plot(sweep_vals, sweep_power, 'r+')
    vmin, vmax = sweep_vals.min(), sweep_vals.max()
    ax.set_xticks(sweep_vals)
    # ax.set_xticks(np.arange(vmin, vmax, ticks_dx), minor=False)
    # ax.set_xlim((0, 50))
    ax.set_ylabel('Power ({})'.format(psd.units))
    ax.set_xlabel(sweep_var_legend)
    ax.grid(True)
    ax.set_title('PSD @ f_input ({})'.format(sig_label))

In [ ]:
%%skip $without_ctx_bursting

f_inputs = [20.0 for i in range(len(analysis_results))]
compare_psd_magnitude('STN_Vm', input_freqs=f_inputs)
compare_psd_magnitude('GPE.proto_Vm', input_freqs=f_inputs)

## 1E: Representative sample

### [main] Rastergram

TODO: use saved figure from individual notebooks

### [alt] Spectrogram

TODO: use saved figure from individual notebooks

# Sec 2 - Role Indirect Pathway

# Sec 3 - Role NMDA-NR2A

In [ ]:
%%javascript
require(["base/js/namespace"],function(Jupyter) {
    Jupyter.notebook.save_checkpoint();
});

In [ ]:
thisfile = 'stn-gpe_sweep_analysis.ipynb'
parent_dir, _ = os.path.split(output_dirs[-1])
!jupyter nbconvert $thisfile --to html --template=toc2 --output-dir=$parent_dir --output $report_filename